In [1]:
def dog_factory(cls, fields):
    try:
        fields = fields.replace(',', '').split()
    except AttributeError:
        pass

    fields = tuple(fields)

    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__=fields,
                     __init__=__init__,
                     __iter__=__iter__,
                     __repr__=__repr__)

    return type(cls, (object,), cls_attrs)


In [5]:
dog = dog_factory('Dog', 'name weight owner')
dog1 = dog(1, 1, 1)
dog1

Dog(name=1, weight=1, owner=1)

In [11]:
import abc


class AutoStorage:
    __cnt = 0

    def __init__(self):
        cls_name = self.__class__.__name__
        index = self.__class__.__cnt
        self.storage_name = f'__{cls_name}#{index}'
        self.__class__.__cnt += 1

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

    def __get__(self, instance, cls):
        if instance is None:
            return self
        return getattr(instance, self.storage_name)


class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)

    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""


class Quantity(Validated):

    def validate(self, instance, value):
        if value >= 0:
            return value
        else:
            raise ValueError


class NoBlank(Validated):

    def validate(self, instance, value):
        value = value.strip()

        if len(value) == 0:
            raise ValueError
        else:
            return value


def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = f'_{type_name}#{key}'
    return cls


@entity
class LineItem:
    description = NoBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price



In [12]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [13]:
dir(raisins)

['_NoBlank#description',
 '_Quantity#price',
 '_Quantity#weight',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'description',
 'price',
 'subtotal',
 'weight']

In [14]:
str.__class__.__name__

'type'

In [15]:
type.__class__

type

In [17]:
class EntityMeta(type):

    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        for key, value in attrs.items():
            if isinstance(value, Validated):
                value.storage_name = f'_{type(value).__name__}#{key}'


class Entity(metaclass=EntityMeta):
    pass

In [22]:

Validated.__bases__

(abc.ABC, __main__.AutoStorage)